Step 01 Grab the Salaries from https://hoopshype.com/salaries/players

In [1]:
import pandas as pd
import numpy as np
import time
#Grab 1991 thru 2023
seasons = np.arange(1991,2024,1)
for season in seasons:
    df = pd.read_html(f'https://hoopshype.com/salaries/players/{season-1}-{season}/')[0]
    df['Year']=season
    df = df.iloc[:,1:5]
    df.columns = ['Player','Actual','Adjusted','Year']
    time.sleep(10)
    if season == 1991: 
        NBAS = df
    else: 
        NBAS = pd.concat([NBAS,df])
NBAS.to_csv('NBA_Salaries_91to23.csv')

KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
import time
#Grab 2024
dfold = pd.read_csv('NBA_Salaries_91to23.csv')
dfold = dfold.iloc[:,1:5]
season = 2024

df = pd.read_html(f'https://hoopshype.com/salaries/players/')[0]

df = df.iloc[:,1:4]
df['Year']=season
df.columns = ['Player','Actual','Adjusted','Year']
time.sleep(10)

NBAS = pd.concat([dfold,df])
NBAS.to_csv('NBA_Salaries_91to24.csv')

Step 02 Grab the Stats from https://www.basketball-reference.com/leagues

In [1]:
import pandas as pd
import numpy as np
import time
seasons = np.arange(1991,2024,1)
for season in seasons:
    df = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{season}_advanced.html')[0]
    df['Year']=season
    time.sleep(10)
    if season == 1991: 
        NBABR = df
    else: 
        NBABR = pd.concat([NBABR,df])

Clean up the NBABR dataset

In [2]:
NBABR01 = NBABR[NBABR['Rk']!='Rk']
NBABR02 = NBABR01.drop_duplicates(subset=['Player','Year'],keep='first')
NBABR02.to_csv('NBABR_Stats_91to23.csv')

Combine Stats with Salaries

In [13]:
NBAstats = pd.read_csv('NBABR_Stats_91to23.csv')
NBASalary = pd.read_csv('NBA_Salaries_91to23.csv')

Repair names in the datasets (remove all * and . in names)(Remove all othe symbols in money columns)

In [14]:
NBAstats['Player']=NBAstats['Player'].str.replace('*','')
NBAstats['Player']=NBAstats['Player'].str.replace('.','')
NBASalary['Player']=NBASalary['Player'].str.replace('.','')
NBASalary['Actual'] = NBASalary['Actual'].str.replace(r'\D+', '', regex=True)
NBASalary['Adjusted'] = NBASalary['Adjusted'].str.replace(r'\D+', '', regex=True)


Convert all weird names

In [16]:
repair = pd.read_csv('NBA2023_repair.csv')
LF = list(repair['tofix'])
LR = list(repair['replacewith'])
NBAstats['Player'] = NBAstats['Player'].replace(LF,LR)
NBASalary['Player'] = NBASalary['Player'].replace(LF,LR)

Sort both datasets by year and then name

In [17]:
stNY = NBAstats[['Player','Year']].sort_values(by=['Year','Player'])
saNY = NBASalary[['Player','Year']].sort_values(by=['Year','Player'])

Merge the datasets together

In [ ]:
dfd = pd.merge(NBAstats,NBASalary,how='left',on=['Player','Year'])
dfa = dfd.dropna(subset=['Actual'])
dfa.to_csv('NBA_Stats_Salary.csv')